#  Data Exploration, Pre-Processing and Augmentation

In [7]:
from dl_utils import *

## 1) Loading the data

In [ ]:
# Set up your Image Generators for the training, validation, and test data

# Don't really know for sure what they mean with this

In [8]:
# Load the datasets as arrays.
path_test = "../data/test"
path_train = "../data/train"
path_val = "../data/val"
X_test, y_test = extract_data(path_test)
X_train, y_train = extract_data(path_train)
X_val, y_val = extract_data(path_val)

## 2) Data exploration

In [ ]:
# Print the training dataset size.

In [ ]:
# Check whether the images always have the same size.

In [ ]:
# Show the distribution of two classes present in the dataset.

In [ ]:
# Plot a few samples from each class with the corresponding label (COVID/normal).

In [ ]:
# Display some statistics on the pixel intensities (e.g., pixel average, global average, pixel standard deviation, global standard deviation, . . . )

In [ ]:
# Repeat the previous steps for the validation and test set:

## 3) Pre-processing

In [ ]:
# Optionally, downsample the images by rescaling their size to 128 × 128 pixels.

In [ ]:
# Implement a normalization strategy, using fixed values, the sample statistics, or the statistics of the training datasets, and test it on a sample from the validation dataset.

## 4) Augmentation

In [ ]:
# Implement three different relevant types of data augmentation.

In [ ]:
# Visualize the transformations on a sample from the training dataset individually and combined.

## 5) Pipeline

In [ ]:
# Implement the pipelines performing pre-processing and augmentation (only for the training dataset) for your next tasks.